In [1]:
!pip install torch torchvision -q

In [2]:
import os
from PIL import Image
import gc

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import average_precision_score


import random
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

### Multi-class probplem

### Resnet training

In [3]:
!nvidia-smi

Thu May 25 10:29:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   34C    P8    14W / 370W |   1602MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:61:00.0 Off |                  N/A |
|  0%   

In [4]:
def clean():
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
clean()

In [6]:
os.environ['CUDA_VISIABLE_DEVICES']='0'

In [7]:
device = torch.device('cuda')

### Efficientnet

In [8]:
from effnet.efficientnet import efficientnet

from utils.trainer import Trainer
from data.multi_class_build_data import build_dataloader

BATCH_SIZE=8

transfrom = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [9]:
criteriation=nn.CrossEntropyLoss()

In [10]:
effnet = efficientnet()
effnet.classifier[1] = nn.Linear(1280, NUM_CLASS)

In [11]:
effnet = effnet.to(device)

In [12]:
optimizer = torch.optim.Adam(effnet.parameters(), lr=2e-4)

### Training 

In [13]:
NAME = 'effnet'

trainer = Trainer(effnet, 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=10,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [14]:
trainer.training()

[1]/[10] Epoch starts
	 Batch train loss: 1.3502655029296875, accuracy 0.5
	 Batch train loss: 1.2573304176330566, accuracy 0.25
	 Batch train loss: 1.4425299167633057, accuracy 0.125
	 Batch train loss: 1.2826417684555054, accuracy 0.25
	 Batch train loss: 1.1898739337921143, accuracy 0.375
[1]/[10] End epoch: train loss: 1.3860933586238222, val loss: 1.3264076586919087
	 Epoch train accuracy: 0.2869338095188141, val accuracy: 0.3425814234016888

[2]/[10] Epoch starts
	 Batch train loss: 1.2382407188415527, accuracy 0.375
	 Batch train loss: 1.3837764263153076, accuracy 0.25
	 Batch train loss: 1.375807762145996, accuracy 0.375
	 Batch train loss: 1.0662955045700073, accuracy 0.625
	 Batch train loss: 1.0794336795806885, accuracy 0.375
[2]/[10] End epoch: train loss: 1.2539523417563925, val loss: 1.146808701223957
	 Epoch train accuracy: 0.38103845715522766, val accuracy: 0.6079613992762364

[3]/[10] Epoch starts
	 Batch train loss: 1.4752596616744995, accuracy 0.25
	 Batch train loss

### Evaluate model

In [18]:
effnet.load_state_dict(torch.load(f'multi_class_output/effnet.pt')['model_state_dict'])
effnet = effnet.to(device)

In [19]:
trainer = Trainer(effnet, 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [20]:
_, _, preds = trainer.val()

In [21]:
prediciton = pd.DataFrame([preds[0].cpu().numpy(), preds[1].cpu().numpy()]).T
prediciton.rename = ['labels', 'predictions']

prediciton.to_csv('multi_class_output/{name}.csv'.format(name=NAME), index=False)

### Multi-label problem

In [22]:
data = pd.read_csv('sample_labels.csv')
data['lables'] = data['Finding Labels'].str.split('|')

labels = []
for lable in data['lables'].values:
    labels.extend(lable)   
    
labels = pd.DataFrame(labels, columns=['labels'])

### Losses

In [24]:
criteriation =  nn.BCELoss()

In [8]:
from utils.trainer import Trainer
from data.multi_label_build_data import build_dataloader
from effnet.blocks import SamePadConv2d
from effnet.efficientnet import efficientnet


BATCH_SIZE=4

transfrom = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(30),
    transforms.ColorJitter(0.5),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS, weights = build_dataloader(transfrom, BATCH_SIZE)

In [33]:
del effnet

In [10]:
effnet = efficientnet()
effnet.classifier[1] = nn.Linear(1280, NUM_CLASS)

effnet.features[0][0] = SamePadConv2d(1, 32, 3, 2)

In [33]:
effnet = effnet.to(device)

In [33]:
optimizer = torch.optim.Adam(effnet.parameters(), lr=1e-4)

NAME = 'effnet'

trainer = Trainer(effnet , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=10,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [39]:
trainer.training()

[1]/[10] Epoch starts
	 Batch train loss: 0.6880797147750854, accuracy 0.31971153846153844
	 Batch train loss: 0.2369377315044403, accuracy 0.43452380952380953
	 Batch train loss: 0.2643577456474304, accuracy 0.5770833333333334
	 Batch train loss: 0.1693349927663803, accuracy 0.675
	 Batch train loss: 0.17834052443504333, accuracy 0.8107142857142857
[1]/[10] End epoch: train loss: 0.2641569006203808, val loss: 0.768161729911972
	 Epoch train accuracy: 0.670088741646833, val accuracy: 0.6765324064543512

[2]/[10] Epoch starts
	 Batch train loss: 0.30318698287010193, accuracy 0.6217261904761905
	 Batch train loss: 0.3578703999519348, accuracy 0.525
	 Batch train loss: 0.20415224134922028, accuracy 0.5875
	 Batch train loss: 0.35156458616256714, accuracy 0.8019688644688645
	 Batch train loss: 0.12280049920082092, accuracy 0.875
[2]/[10] End epoch: train loss: 0.22381317974665227, val loss: 0.4186122650364713
	 Epoch train accuracy: 0.6895479717441803, val accuracy: 0.6697175925142027

[3]

### Focal

In [69]:
class WeightedMultilabel(nn.Module):  
    def __init__(self, weights: torch.Tensor):  
        super(WeightedMultilabel, self).__init__()  
        self.cerition = nn.BCELoss(reduction='none')  
        self.weights = weights  
  
    def forward(self, outputs, targets):  
        loss = self.cerition(outputs, targets)  
        return (loss * self.weights).mean()  

In [71]:
weights = weights.to(device)

In [72]:
criteriation = WeightedMultilabel(weights)

In [73]:
del effnet

In [74]:
effnet = efficientnet()
effnet.classifier[1] = nn.Linear(1280, NUM_CLASS)
effnet.features[0][0] = SamePadConv2d(1, 32, 3, 2)

In [75]:
effnet = effnet.to(device)

In [76]:
optimizer = torch.optim.Adam(effnet.parameters(), lr=1e-4)

In [77]:
NAME = 'effnet_focal'

trainer = Trainer(effnet , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=10,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [78]:
trainer.training()

[1]/[10] Epoch starts
	 Batch train loss: 51.64650156964188, accuracy 0.30862470862470864
	 Batch train loss: 15.696557153410076, accuracy 0.5625
	 Batch train loss: 6.025534986873829, accuracy 0.6083333333333334
	 Batch train loss: 17.786056907131204, accuracy 0.4461399711399711
	 Batch train loss: 6.034664337969898, accuracy 0.49603174603174605
[1]/[10] End epoch: train loss: 9.355495370521558, val loss: 6.923498458618368
	 Epoch train accuracy: 0.6761730445635123, val accuracy: 0.6827294329468727

[2]/[10] Epoch starts
	 Batch train loss: 8.03415521270672, accuracy 0.36339285714285713
	 Batch train loss: 2.78274725960867, accuracy 0.7847222222222222
	 Batch train loss: 7.686811124783851, accuracy 0.4801587301587302
	 Batch train loss: 5.804737982952142, accuracy 0.5857142857142857
	 Batch train loss: 4.362739393921555, accuracy 0.4895833333333333
[2]/[10] End epoch: train loss: 5.947070115186447, val loss: 4.803368586107251
	 Epoch train accuracy: 0.6897559254168593, val accuracy: 0

### Evaluate model

In [16]:
criteriation =  nn.BCELoss()
NAME='effnet'

In [17]:
effnet.load_state_dict(torch.load(f'multi_label_output/effnet.pt')['model_state_dict'])
effnet = effnet.to(device)

In [18]:
trainer = Trainer(effnet , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [19]:
_, _, preds = trainer.val()

In [21]:
pd.DataFrame(preds[0].cpu().numpy()).to_csv('multi_label_output/true_{name}.csv'.format(name=NAME), index=False)
pd.DataFrame(preds[1].cpu().numpy()).to_csv('multi_label_output/pred_{name}.csv'.format(name=NAME), index=False)